# LAMMPS input file

In [152]:
T = 1400
xyrate_1_per_s = 1e6

#pair_coeff = "eam/fs"
#potential = "Cu1.eam.fs"
pair_coeff = "eam/alloy"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"

in_nemd_script = f"""# Sample LAMMPS input script for viscosity of FCC solid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable	    x equal 14
variable	    y equal 14
variable        z equal 14

variable	    a equal 3.70254
variable        T equal {T}
variable        P equal 1 

variable        xyrate_1_per_s equal {xyrate_1_per_s}
variable        xyrate_1_per_ps equal v_xyrate_1_per_s/1.0e12

# Set log file to include temperature
log             log_T{T}.lammps

# Problem setup
units		    metal
dimension	    3
boundary        p p p
atom_style	    atomic

lattice         fcc ${{a}}
region          simbox prism 0 $x 0 $y 0 $z 0.0 0.0 0.0 units lattice
create_box      1 simbox
create_atoms    1 box

pair_style      {pair_coeff}
pair_coeff      * * {potential} Cu
mass            1  63.55

# 1. Equilibration run
velocity        all create $T 97287
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)

thermo          1000
#dump            equilibration all custom 50 dump.equilibration_T{T} id type x y z vx vy vz 
run	            1000000

#undump         equilibration
unfix  	        1
reset_timestep  0

# 2. Turn on NEMD shear and equilibrate some more
#variable	    xyrate equal ${{srate}}/ly # Shear rate defined relative to perpendicular dimension (1/ps)

fix		        1 all nvt/sllod temp $T $T $(100.0*dt)

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${{xyrate_1_per_ps}} remap v

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

#dump            nemd_equilibration all custom 50 dump.nemd_equilibration_T{T} id type x y z vx vy vz
run		        1000000
#undump          nemd_equilibration
reset_timestep  0

# 3. Data gathering run
# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center 0.05 units reduced
fix		        3 all ave/chunk 10 100 1000 layers_center vx file profile_center_T{T}.nemd

# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_xyrate_1_per_ps)*0.0001 # Computed every timestep. Convert bar.ps to mPa.s # Check this

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average averages the average values computed every 1000 timesteps.
fix             vprint all print 10 \"${{timestep}} ${{visc}}\" file visc_output_T{T}.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 1000 file visc_run_avg_T{T}.txt

thermo          1000
thermo_style	custom step temp etotal press pxy ly v_visc f_vave
thermo_modify	temp tilt

#dump	        data_gathering all custom 100 dump.data_gathering_T{T} id type x y z
dump            id all atom 500 dump.data_gathering_T{T}

run		        5000000
"""

# Write LAMMPS input file

In [153]:
import os

xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 
path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{T}K/{xyrate_str}/"

os.makedirs(path, exist_ok=True)
with open(path + "in.nemd", "w") as f:
    f.write(in_nemd_script)

# Job script

In [154]:
job_script = """#!/bin/bash
#SBATCH --job-name=Bridges-2
#SBATCH -A dmr190011p
#SBATCH -p RM-512
#SBATCH -N 2
#SBATCH --ntasks-per-node 64
#SBATCH -t 48:00:00

module purge
module load intelmpi/2021.3.0-intel2021.3.0
module load gcc/10.2.0
module load cuda/11.7.1
module load python

echo "SLURM_NTASKS: " $SLURM_NTASKS

ulimit -s unlimited
export OMP_NUM_THREADS=1

mpirun -n $SLURM_NTASKS /opt/packages/LAMMPS/lammps-23Jun2022/build/lmp -sf omp -pk omp $OMP_NUM_THREADS -in in.nemd
"""

with open(path + "job.sh", "w") as f:
    f.write(job_script)

# Write job script

In [155]:
# Copy the selected potential file to the simulation path
import shutil
shutil.copy(f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential}", path + potential)

'/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04_eam_alloy/x=1/1400K/1e6/CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy'

# Submit the job

In [156]:
import os
import subprocess

cwd = os.getcwd()
os.chdir(path)
try:
    subprocess.run(["sbatch", "job.sh"], check=True)
finally:
    os.chdir(cwd)

Submitted batch job 33269480


# Plot the velocity profile

In [157]:
import plotly.graph_objects as go

# Function to vx for a specific timestep using Plotly
def plot_vx(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            yaxis_title='y/l<sub>y</sub>',
            xaxis_title='vx',
            template='plotly_white',
            width=800,
            height=600 
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [159]:
T = 1400
xyrate_1_per_s = 1e10

#pair_coeff = "eam/fs"
#potential = "Cu1.eam.fs"
pair_coeff = "eam/alloy"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/old/{T}K/{xyrate_str}/"
velocity_profile_file = path + f'profile_center_T{T}.nemd'

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open(velocity_profile_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

In [160]:
time = 1e6
plot_vx(time)

In [170]:
import numpy as np
import plotly.graph_objects as go

# Get all unique chunk indices (assume all timesteps have the same chunking)
first_timestep = next(iter(chunk_data))
num_chunks = len(chunk_data[first_timestep]['chunk'])
chunk_indices = np.array(chunk_data[first_timestep]['chunk'])
coords = np.array(chunk_data[first_timestep]['coord1'])

# Stack vx for each chunk across all timesteps
vx_matrix = []
for t in chunk_data:
    vx_matrix.append(chunk_data[t]['vx'])
vx_matrix = np.array(vx_matrix)  # shape: (num_timesteps, num_chunks)

# Average vx for each chunk over all timesteps
avg_vx_per_chunk = np.mean(vx_matrix, axis=0)

# Plot y/ly (coords) vs average vx
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=avg_vx_per_chunk,
    y=coords,
    mode='lines+markers',
    name='Average vx (spatial)'
))
fig.update_layout(
    title='Spatially Averaged Velocity Profile: y/ly vs. Average vx',
    yaxis_title='y / ly',
    xaxis_title='Average vx',
    template='plotly_white',
    width=800,
    height=500
)
fig.show()

In [165]:
chunk_data[1000]

{'chunk': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20],
 'coord1': [0.025,
  0.075,
  0.125,
  0.175,
  0.225,
  0.275,
  0.325,
  0.375,
  0.425,
  0.475,
  0.525,
  0.575,
  0.625,
  0.675,
  0.725,
  0.775,
  0.825,
  0.875,
  0.925,
  0.975],
 'ncounts': [543.52,
  553.56,
  552.37,
  549.52,
  546.58,
  551.68,
  558.09,
  541.92,
  547.19,
  551.82,
  545.4,
  546.77,
  548.48,
  545.05,
  552.47,
  548.24,
  542.26,
  554.71,
  551.1,
  545.27],
 'vx': [-0.308324,
  -0.30511,
  -0.271733,
  -0.324403,
  -0.244942,
  -0.16806,
  -0.120399,
  -0.0636662,
  -0.0543212,
  -0.0473553,
  -0.0212461,
  0.0844585,
  0.060956,
  0.134413,
  0.192333,
  0.211703,
  0.150575,
  0.222497,
  0.356506,
  0.158052]}

# Understand viscosity averaging

In [ ]:
import numpy as np

# Read viscosity_output.txt into numpy array ignoring the first line
data = np.loadtxt('visc_output.txt', skiprows=1)

# Reshape the second column to group every 100 values
second_column = data[:, 1][1:]
reshaped = second_column[:].reshape(-1, 100)

# Compute the mean for each group
calculated_averages = reshaped.mean(axis=1)

# Compute the running average
calculated_running_averages = np.cumsum(calculated_averages) / np.arange(1, len(calculated_averages) + 1)

In [ ]:
# Read the viscosity_averages.txt file into a numpy array ignoring the first two lines
averages = np.loadtxt('visc_run_avg.txt', skiprows=2)
running_averages = averages[:, 1]

In [ ]:
# Compare the LAMMPS calculated running average with the one we computed
np.allclose(running_averages, calculated_running_averages, atol=1e-6)

True

In [ ]:
running_averages

array([16079. , 16445.2, 17807.3, 19643. , 21473.3, 22648.4, 23349.3,
       23808.7, 23684.4, 24127.5, 24009.8, 24027.2, 24556.8, 24335.6,
       24227.8, 24033.8, 23303.3, 23384.4, 22751.1, 22989.7, 22714.9,
       22870.1, 23244. , 23351.7, 23459. , 23637.7, 23384.5, 23208.6,
       23021.6, 22725.3, 22310.6, 22236.5, 22256.8, 22124.5, 21696.9,
       21601.3, 21806.8, 21569.2, 21515.8, 21487.8, 21481.1, 21410.8,
       21304. , 21230. , 21232.5, 21120.3, 21076.6, 20987.3, 20895.8,
       20922.5])

## Plot viscosity vs. temperature

In [ ]:
viscosity_zhou = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

#pair_coeff = "eam/fs"
#potential = "Cu1.eam.fs"
potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_zhou.append(viscosity_value)
        

In [ ]:
viscosity_zhou = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_zhou.append(viscosity_value)
        

In [ ]:
viscosity_mendelev = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

potential = "Cu1.eam.fs"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Get the last line of the file
    with open(log_file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        # Extract the viscosity value from the last line
        viscosity_value = float(last_line.split()[1])
        viscosity_mendelev.append(viscosity_value)


In [ ]:
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Create a scatter plot for viscosity vs. temperature
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=temperatures,
    y=viscosity_zhou,
    mode='markers',
    name='Zhou et al. (2004)',
    marker={'size': 10}
))

fig.add_trace(go.Scatter(
    x=temperatures,
    y=viscosity_mendelev,
    mode='markers',
    name='Mendelev et al. (2008)',
    marker={'size': 10}
))

fig.add_vline(x=1358, line_dash='dash', line_color='black')
fig.update_layout(showlegend=True)
fig.update_yaxes(range=[1.5, 6.0])
fig.update_xaxes(range=[1200, 2050])
plot_format(fig, xtitle='Temperature (K)', ytitle='Viscosity (mPa·s)')
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

viscosity = []
temperatures = [1400, 1550, 1700, 1850, 2000]
xyrate_1_per_s = 1e10

potential = "Cu1.eam.fs"
#potential = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"
xyrate_str = "{:.0e}".format(xyrate_1_per_s).replace("+0", "").replace("+", "")
potential_str = potential.replace('.', '_') 

fig = go.Figure()

for temperature in temperatures:
    path = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Cu/{potential_str}/x=1/{temperature}K/{xyrate_str}/"
    log_file = path + f'visc_run_avg_T{temperature}.txt'
    # Load data, skipping the first 2 header lines
    data = np.loadtxt(log_file, skiprows=2)
    timesteps = data[:, 0]
    viscosity = data[:, 1]
    fig.add_trace(go.Scatter(
        x=timesteps,
        y=viscosity,
        mode='lines',
        name=f'{temperature} K'
    ))

fig.update_layout(
    title='Viscosity vs. Timestep for Each Temperature (Mendelev et al. 2008)',
    xaxis_title='Timestep (fs)',
    yaxis_title='Viscosity (mPa·s)',
    template='plotly_white',
    showlegend=True
)
plot_format(fig, xtitle='Timestep (fs)', ytitle='Viscosity (mPa·s)')
fig.show()

In [ ]:
data

array([[1.00000e+03, 1.07531e+00],
       [2.00000e+03, 1.33660e+00],
       [3.00000e+03, 1.00178e+00],
       ...,
       [4.99800e+06, 2.03858e+00],
       [4.99900e+06, 2.03895e+00],
       [5.00000e+06, 2.03926e+00]])